In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
import os
import numpy as np

from collections import Counter

# Loads each whole scenario as a sheet (For Later)

# Loads and seperates each car into its own sheet

In [3]:
start_folder = 'AllCSVs'
csvs = glob.glob(r'{}\*.csv'.format(start_folder))
df_list = []

for c in csvs:
    df = pd.read_csv(c)
    for i in np.unique(df.sender):
        df_list.append(df[df.sender.isin([i])].copy())
    #     df[df.ID.isin([subs[0]])&df.Vigil.isin(vigils[0])].copy()
print(len(df_list))
# df_list[0]

47539


In [5]:
df_list[10]

,type,time,sender,attackerType,messageID,pos0,pos1,pos2,spd_noise0,spd_noise1,spd_noise2,pos_noise0,pos_noise1,pos_noise2,spd0,spd1,spd2
116,4,10816.639813,67,0,2711,3643.069004,5197.317764,1.895,0.0,0.0,0.0,0.0,0.0,0.0,-4.616352,29.593556,0.0
124,4,10817.639813,67,0,2879,3638.460473,5226.861177,1.895,0.0,0.0,0.0,0.0,0.0,0.0,-4.617208,29.599043,0.0
132,4,10818.639813,67,0,3060,3633.852011,5256.404157,1.895,0.0,0.0,0.0,0.0,0.0,0.0,-4.611753,29.564076,0.0
139,4,10819.639813,67,0,3220,3629.242940,5285.951033,1.895,0.0,0.0,0.0,0.0,0.0,0.0,-4.617850,29.603158,0.0
146,4,10820.639813,67,0,3341,3624.633830,5315.498164,1.895,0.0,0.0,0.0,0.0,0.0,0.0,-4.609175,29.547547,0.0
152,4,10821.639813,67,0,3452,3621.123040,5345.211319,1.895,0.0,0.0,0.0,0.0,0.0,0.0,-3.437161,29.782636,0.0
159,4,10822.639813,67,0,3571,3617.693503,5374.927891,1.895,0.0,0.0,0.0,0.0,0.0,0.0,-3.426757,29.692482,0.0
166,4,10823.639813,67,0,3694,3614.263615,5404.647503,1.895,0.0,0.0,0.0,0.0,0.0,0.0,-3.435915,29.771840,0.0
173,4,10824.639813,67,0,3817,3610.834941,5434.356603,1.895,0.0,0.0,0.0,0.0,0.0,0.0,-3.437423,29.784902,0.0
180,4,10825.639813,67,0,3946,3607.407462,5464.055341,1.895,0.0,0.0,0.0,0.0,0.0,0.0,-3.430100,29.721452,0.0


# Note: you would find and organize the cars that are the closest right here and more the dataframe with that info now (Ex: for a given row with Car7 look N rows up and pick different carX with closest POS (Not Sped))

## ****.attackerType can be 1,2,4,8,16 and are different types of attacks May need to seperate out

In [6]:
df_attacker = []
df_normal = []
window_size = 11
overlap_size = 10

for j in df_list:
    if j.attackerType.any() == 0:
        j = j.iloc[:,[1,2,5,6,14,15]]
        for start_idx in range(0, len(j)-window_size, window_size - overlap_size):
            end_idx = start_idx + window_size
            df_chunk = j.iloc[start_idx:end_idx]
#             display(df_chunk)
            df_normal.append(df_chunk)
        
    
    elif j.attackerType.any() == 1:
        j = j.iloc[:,[5,6,14,15]]
        df_attacker.append(j)

# df_normal = np.asarray(df_normal)
# df_attacker = np.asarray(df_attacker)

df_normal[0]

,time,sender,pos0,pos1,spd0,spd1
1,10800.437587,7,3609.398494,5446.803264,-3.534058,30.622234
8,10801.437587,7,3605.873370,5477.348089,-3.533871,30.620620
14,10802.437587,7,3603.260815,5507.988690,-2.516747,30.722842
20,10803.437587,7,3600.748307,5538.659784,-2.513009,30.677207
26,10804.437587,7,3598.236829,5569.318316,-2.512296,30.668505
32,10805.437587,7,3595.724619,5599.985772,-2.510090,30.641585
39,10806.437587,7,3593.213558,5630.639206,-2.514574,30.696319
45,10807.437587,7,3591.439053,5661.354795,-1.284169,30.756852
52,10808.437587,7,3590.156264,5692.078590,-1.287250,30.830647
59,10809.437587,7,3588.873085,5722.811723,-1.281963,30.704012


In [8]:
len(df_normal)
df_normal[12]

,time,sender,pos0,pos1,spd0,spd1
13,10802.392785,13,3590.806174,5619.665087,-3.174214,38.748286
19,10803.392785,13,3588.263022,5658.409084,-1.620634,38.775840
25,10804.392785,13,3586.641595,5697.203884,-1.621226,38.789993
31,10805.392785,13,3585.019604,5736.012187,-1.621550,38.797751
38,10806.392785,13,3584.415782,5774.867693,-0.037708,38.905668
44,10807.392785,13,3584.378132,5813.713295,-0.037663,38.860032
51,10808.392785,13,3584.340460,5852.582601,-0.037692,38.889646
58,10809.392785,13,3584.627648,5891.452273,1.655924,38.797120
65,10810.392785,13,3586.284671,5930.275148,1.658909,38.867065
73,10811.392785,13,3589.162657,5969.034509,2.882762,38.770413


# Next take df_normal and for each element in it put 10 rows in X and 1 row into Y (Convert to numpy) For Model 1

In [9]:
X = []
y = []
for df in df_normal:
    X.append(np.asarray(df.iloc[0:10,2:]))
    y.append(np.asarray(df.iloc[10,2:]))
X = np.asarray(X)
y = np.asarray(y)
np.save(r'{}\X.npy'.format(start_folder), X)
np.save(r'{}\y.npy'.format(start_folder), y)

In [15]:
X1 = np.load(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\{}\X.npy'.format(start_folder))
X1.shape

(1196218, 10, 4)

In [16]:
X1[0]

array([[ 3.60939849e+03,  5.44680326e+03, -3.53405773e+00,
         3.06222342e+01],
       [ 3.60587337e+03,  5.47734809e+03, -3.53387139e+00,
         3.06206197e+01],
       [ 3.60326082e+03,  5.50798869e+03, -2.51674686e+00,
         3.07228422e+01],
       [ 3.60074831e+03,  5.53865978e+03, -2.51300856e+00,
         3.06772074e+01],
       [ 3.59823683e+03,  5.56931832e+03, -2.51229565e+00,
         3.06685046e+01],
       [ 3.59572462e+03,  5.59998577e+03, -2.51009046e+00,
         3.06415851e+01],
       [ 3.59321356e+03,  5.63063921e+03, -2.51457414e+00,
         3.06963190e+01],
       [ 3.59143905e+03,  5.66135479e+03, -1.28416894e+00,
         3.07568515e+01],
       [ 3.59015626e+03,  5.69207859e+03, -1.28725005e+00,
         3.08306467e+01],
       [ 3.58887309e+03,  5.72281172e+03, -1.28196275e+00,
         3.07040115e+01]])

In [17]:
X1[30]

array([[ 3.82936674e+03,  5.26232798e+03,  5.06028439e+00,
        -3.48143023e+00],
       [ 3.83537483e+03,  5.25819446e+03,  6.94068123e+00,
        -4.77512637e+00],
       [ 3.84336042e+03,  5.25280902e+03,  9.14090422e+00,
        -6.02007260e+00],
       [ 3.85358731e+03,  5.24721513e+03,  1.20621312e+01,
        -4.81824196e+00],
       [ 3.86744993e+03,  5.24855892e+03,  1.47329413e+01,
         2.08237175e+00],
       [ 3.88184276e+03,  5.25059322e+03,  1.47054407e+01,
         2.07848477e+00],
       [ 3.89622224e+03,  5.25262563e+03,  1.46322612e+01,
         2.06814150e+00],
       [ 3.91059872e+03,  5.25465762e+03,  1.46733947e+01,
         2.07395536e+00],
       [ 3.92499009e+03,  5.25669171e+03,  1.47283264e+01,
         2.08171948e+00],
       [ 3.93940498e+03,  5.25872913e+03,  1.47675056e+01,
         2.08725711e+00]])